In [2]:
# base code for jupyter integration

import sys, os, django

BASE_DIR = os.path.dirname(os.path.abspath('../../agrai/'))

sys.path.insert(0, BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "agrai.settings")

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

In [5]:
from core.models import *

# necesitamos un conjunto de fechas:
# fechas = ["2020-01-01", "2020-01-31", "2020-01-31", "2020-01-31"]

queryset = Mirar_Indice.objects.values('fecha').distinct()

fechas = [ x['fecha'] for x in queryset ]

print(fechas)

[datetime.date(2022, 6, 2), datetime.date(2022, 6, 17), datetime.date(2022, 6, 7), datetime.date(2022, 6, 27), datetime.date(2022, 6, 22), datetime.date(2022, 6, 12)]


In [4]:
# obtener las cuatro ultimas fechas:

# en nuestro caso estamos reemplazado el proceso de calibrado de estas fechas, por la búsqeda de las cuatro últimas que se encuentren en la base de datos.

In [8]:
from core.models import *

import pandas as pd

def statistic_indices(indices = ['ndvi', 'ndre'], func=np.mean):

    df = pd.DataFrame()

    col_ids = []

    for p in Parcela.objects.all():

        col_ids.append(p.idx)

    df['IDX'] = col_ids

    # para cada indice:
    for ind in indices:

        indice_p = Indice.objects.get(nombre=ind) # if type(indice_p) == str else indice_p

        for fecha in fechas: 

            col_data = []

            # por cada iteración de esto tienes una fila en el df:
            for p in Parcela.objects.all():

                # la media de todos sus índices
                p_indices = Pixel.objects.filter(parcela=p)

                list_values = []

                for p_itr in p_indices:

                    qs =  Mirar_Indice.objects.get(

                        pixel = p_itr, 
                        indice = indice_p, 
                        fecha = fecha # fecha para la columna:
                    )
                    
                    import math

                    if ( not math.isnan(qs.valor) ):

                        list_values.append(qs.valor)

                # estadistico:añadir columna
                
                res = func(list_values)

                col_data.append(res)

            df[func.__name__ + '_' + ind +'_' +str(fecha)] = col_data

            # valor a la tabla

    return df

In [9]:
df1 = statistic_indices(['ndvi', 'ndre'], np.mean)

df2 = statistic_indices(['ndvi', 'ndre'], np.sum)

df3 = pd.concat([df1, df2], axis=1)

df3

,IDX,mean_ndvi_2022-06-02,mean_ndvi_2022-06-17,mean_ndvi_2022-06-07,mean_ndvi_2022-06-27,mean_ndvi_2022-06-22,mean_ndvi_2022-06-12,mean_ndre_2022-06-02,mean_ndre_2022-06-17,mean_ndre_2022-06-07,...,sum_ndvi_2022-06-07,sum_ndvi_2022-06-27,sum_ndvi_2022-06-22,sum_ndvi_2022-06-12,sum_ndre_2022-06-02,sum_ndre_2022-06-17,sum_ndre_2022-06-07,sum_ndre_2022-06-27,sum_ndre_2022-06-22,sum_ndre_2022-06-12
0,3942,0.007476,0.212770,0.180844,-0.007705,0.038478,0.219896,-0.017073,0.151714,0.123746,...,12.839933,-0.547055,2.731913,15.612608,-1.212181,10.771678,8.785966,-1.924856,-0.259414,10.886635
1,4053,0.008709,0.202058,0.165046,-0.005460,-0.019584,0.198812,-0.007738,0.134886,0.106043,...,32.018860,-1.059211,-3.799293,38.569600,-1.501124,26.167940,20.572346,-4.967695,-7.591245,24.986805
2,4196,0.204576,0.192939,0.156745,0.001065,-0.011448,0.186642,0.226347,0.128658,0.095656,...,15.047543,0.102206,-1.099016,17.917662,21.729303,12.351145,9.183014,-2.268778,-4.328455,11.240659
3,4422,-0.001838,0.198717,0.181343,-0.009038,-0.002894,0.215428,-0.036878,0.132829,0.113879,...,10.336535,-0.515191,-0.164956,12.279409,-2.102046,7.571269,6.491099,-1.605132,-1.682397,7.757051
4,4424,-0.046762,0.205482,0.185708,-0.010250,-0.005377,0.218237,-0.094255,0.145178,0.124556,...,6.871180,-0.379265,-0.198932,8.074769,-3.487440,5.371600,4.608560,-1.066419,-1.157790,5.483689
5,3882,0.000592,0.224865,0.189626,-0.004974,0.062246,0.233515,-0.018172,0.159256,0.129561,...,10.619075,-0.278543,3.485752,13.076824,-1.017618,8.918353,7.255410,-1.365950,2.310275,8.944062
6,4732,0.256245,0.177747,0.141133,-0.000639,-0.054281,0.170076,0.252110,0.114163,0.080882,...,5.080800,-0.023017,-1.954113,6.122730,9.075973,4.109874,2.911769,-1.106963,-3.733478,3.668665
7,3553,0.036759,0.119855,0.116412,-0.006666,-0.002644,0.106559,0.028544,0.076208,0.070953,...,8.730864,-0.499949,-0.198277,7.991947,2.140790,5.715596,5.321453,-1.985539,-2.377342,4.499377
8,3582,0.115627,0.216081,0.176353,0.004865,-0.002795,0.209227,0.091589,0.144046,0.110570,...,10.052107,0.277332,-0.159307,11.925953,5.220590,8.210635,6.302486,-1.276941,-2.043705,7.499418
9,3914,0.172558,0.315941,0.291676,0.005314,-0.028690,0.364501,0.127867,0.210205,0.200281,...,37.626256,0.685459,-3.701053,47.020692,16.494828,27.116386,25.836284,-2.855642,-8.192364,31.391749


In [10]:
#exportar df _>

import csv

df3.to_csv('datos_df3.csv', index=False)